In [ ]:
from labo_instruments import SR830
import pandas as pd
import numpy as np
import pyvisa as visa
import time
import matplotlib.pyplot as plt
import os

In [3]:
help(SR830)

Help on class SR830 in module labo_instruments.sr830:

class SR830(builtins.object)
 |  SR830(resource)
 |
 |  Clase para el manejo amplificador Lockin SR830 usando PyVISA de interfaz
 |
 |  Methods defined here:
 |
 |  __del__(self)
 |      Finaliza la conexión con el Lock-in SR830 y desbloquea el panel frontal.
 |
 |      Side Effects:
 |          - Envía el comando "LOCL 0" para habilitar el control manual del equipo.
 |          - Cierra la conexión VISA.
 |
 |  __init__(self, resource)
 |      Inicializa la conexión con el Lock-in Amplifier SR830 mediante PyVISA.
 |
 |      Args:
 |          resource (str): Dirección del recurso VISA del instrumento (por ejemplo, "GPIB0::8::INSTR").
 |
 |      Side Effects:
 |          - Bloquea el panel frontal del equipo para evitar interacción manual.
 |          - Obtiene la escala y constante de tiempo actuales del equipo.
 |
 |  auto_scale(self)
 |      Ajusta automáticamente la escala del Lock-in para optimizar la medición de la magnitud R.

In [4]:
rm = visa.ResourceManager()
print(rm.list_resources())

()


In [ ]:
fungen = rm.open_resource('USB0::0x0699::0x0346::C034166::INSTR')
print(fungen.query('*IDN?'))
lockin = SR830('GPIB0::7::INSTR')


TEKTRONIX,AFG3021B,C034166,SCPI:99.0 FV:3.1.1



In [ ]:
X = []
Y = []
V = []

for i in np.linspace(-1,1,50):
    lockin.set_aux_out(1,i)  
    lockin.auto_scale()
    time.sleep(0.5)
    medicion = lockin.get_medicion()
    print(medicion)
    
    V.append(i)
    X.append(medicion[0])
    Y.append(medicion[1])
    

[2.30785e-11, -1.44383e-11]
[2.41301e-11, -1.39267e-11]
[2.55512e-11, -1.40688e-11]
[2.4528e-11, -1.49499e-11]
[2.55512e-11, -1.50068e-11]
[2.70007e-11, -1.57457e-11]
[2.62902e-11, -1.40404e-11]
[2.61197e-11, -1.48931e-11]
[2.6546e-11, -1.50352e-11]
[2.63755e-11, -1.50068e-11]
[2.49828e-11, -1.56605e-11]
[2.31354e-11, -1.62858e-11]
[2.29649e-11, -1.48362e-11]
[2.18848e-11, -1.66552e-11]
[2.06059e-11, -1.60868e-11]
[1.77068e-11, -1.60015e-11]
[1.55752e-11, -1.52057e-11]
[1.23919e-11, -1.48647e-11]
[7.81603e-12, -1.33014e-11]
[3.58115e-12, -1.31025e-11]
[-8.24233e-13, -1.3074e-11]
[-6.93492e-12, -1.04877e-11]
[-1.36709e-11, -1.12266e-11]
[-1.92132e-11, -1.04877e-11]
[-2.58923e-11, -9.12342e-12]
[-3.40779e-11, -8.58341e-12]
[-4.20928e-11, -8.4413e-12]
[-4.96245e-11, -8.0434e-12]
[-5.7071e-11, -6.73597e-12]
[-6.57967e-11, -5.42857e-12]
[-7.37548e-11, -4.66119e-12]
[-8.22813e-11, -4.37697e-12]
[-9.06089e-11, -2.70008e-12]
[-9.86522e-11, -2.92745e-12]
[-1.08742e-10, -2.38744e-12]
[-1.16359e-

In [ ]:
X = np.array(X)*(-1)
Y = np.array(Y)
V = np.array(V)

plt.figure()
plt.plot(V,X)
plt.grid()

NameError: name 'X' is not defined

In [135]:
data = pd.DataFrame({'X':X,
                     'Y':Y,
                     'V':V})

In [ ]:
ruta_salida = os.path.join(os.path.dirname(__file__), "Data", "voltaje_verde.csv")
data.to_csv(ruta_salida, index=False)